# Vlaamse codex notebook

Deze notebook is gemaakt vanuit de opleiding PXL business architect AI. Voor de case rond de **Vlaamse Codex**

Teamleden: Koen, Charlotte, Kim

Coach: Tim

Code door Koen Mekers (functioneel analist, geen dev dus de code is niet helemaal optimaal)

Special thanks to: 



# Deze cell is de scraper cell. Hier wordt de volledige codex ingehaald in Jsonformaat.
Er is een mogelijkheid om de volledige codex in te halen, of te kiezen voor een select aantal. Duurt ca. 40 min 

In [ ]:
# Vlaamse codex API notebook
import requests
import os
import time 
from datetime import datetime
import json
from tqdm import tqdm # Voor progress bar (I know, it's awesome)

# Configuratie
OUTPUT_DIR = "codexjson"
ERROR_LOG = "error_log.txt"
BASE_URL = "https://codex.opendata.api.vlaanderen.be/api/WetgevingDocument"

# Zorg dat de output directory bestaat
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Maak een session object aan voor hergebruik van verbindingen => epic scrapesnelheid
SESSION = requests.Session()

def log_error(message):
    """Log een error met timestamp."""
    with open(ERROR_LOG, 'a', encoding='utf-8') as f: # encoding toegevoegd
        f.write(f"{datetime.now()} - {message}\n")

def fetch_document(session, doc_id, retries=3, error_delay=1): # error_delay voor algemene fouten
    """
    Haal de JSON-data op voor een gegeven document-ID.
    Probeert maximaal 'retries' keer bij falen.
    Bij status 429 (rate limit) wordt dit gelogd, maar niet lang gewacht.
    """
    url = f"{BASE_URL}/{doc_id}/VolledigDocument"

    for attempt in range(1, retries + 1):
        try:
            response = session.get(url, timeout=20) # Timeout voor het request

            if response.status_code == 200:
                try:
                    return response.json()
                except json.JSONDecodeError as e_json:
                    log_error(f"Attempt {attempt}: JSONDecodeError for document {doc_id} (status 200). Error: {str(e_json)}. Response text: {response.text[:200]}")
                    time.sleep(error_delay) # Korte wacht bij JSON parse error
            elif response.status_code == 429: # Rate limit, gaat niet gebeuren, maar toch.
                log_error(f"Attempt {attempt}: Rate limit (429) hit for document {doc_id}. API returned 429, proceeding with minimal delay.")
                time.sleep(0.1) # Minimale pauze
            else: # Andere HTTP errors (404, 500, etc.)
                log_error(f"Attempt {attempt}: HTTP {response.status_code} for document {doc_id}. Response: {response.text[:200]}. Retrying after {error_delay}s.")
                time.sleep(error_delay)
        
        except requests.exceptions.Timeout:
            log_error(f"Attempt {attempt}: Timeout for document {doc_id}. Retrying after {error_delay}s.")
            time.sleep(error_delay)
        except requests.exceptions.ConnectionError as e_conn:
            log_error(f"Attempt {attempt}: ConnectionError for document {doc_id}: {str(e_conn)}. Retrying after {error_delay}s.")
            time.sleep(error_delay)
        except requests.exceptions.RequestException as e_req: # Vangt andere requests-gerelateerde exceptions
            log_error(f"Attempt {attempt}: RequestException for document {doc_id}: {str(e_req)}. Retrying after {error_delay}s.")
            time.sleep(error_delay)
        except Exception as e_generic:
             log_error(f"Attempt {attempt}: Unexpected generic exception for document {doc_id}: {type(e_generic).__name__} - {str(e_generic)}. Retrying after {error_delay}s.")
             time.sleep(error_delay)

    log_error(f"Failed to fetch document {doc_id} after {retries} attempts.")
    return None

def get_documents(session, full_codex=False, start_id=None, num_docs=None):
    """
    Verwerkt documenten:
      - full_codex=True: haal de volledige lijst op via paginatie (met top/skip).
      - anders: verwerk een ID-range.
    """
    if full_codex:
        documents_list_meta = []
        top = 200  # Aantal resultaten per pagina
        skip = 0   # Offset
        page_num = 1

        print("Fetching document metadata list...")
        while True:
            params = {"top": top, "skip": skip}
            request_url = f"{BASE_URL}?top={top}&skip={skip}"
            try:
                # Gebruik session & timeout voor het ophalen van de lijst
                response = session.get(BASE_URL, params=params, timeout=30)
                response.raise_for_status()  # Genereert een HTTPError voor 4xx/5xx status codes
                data = response.json()
            except requests.exceptions.HTTPError as e:
                print(f"Stopping. HTTP error fetching document list (page {page_num}, skip {skip}): {e}")
                log_error(f"HTTPError fetching document list: {e}. URL: {e.request.url if e.request else request_url}")
                break 
            except requests.exceptions.Timeout as e:
                print(f"Stopping. Timeout fetching document list (page {page_num}, skip {skip}): {e}")
                log_error(f"Timeout fetching document list: {e}. URL: {request_url}")
                break
            except requests.exceptions.RequestException as e:
                print(f"Stopping. Error fetching document list (page {page_num}, skip {skip}): {e}")
                log_error(f"RequestException fetching document list: {e}. URL: {request_url}")
                break
            except json.JSONDecodeError as e:
                print(f"Stopping. Error decoding JSON from document list (page {page_num}, skip {skip}): {e}")
                log_error(f"JSONDecodeError fetching document list: {e}. Response text: {response.text[:200] if 'response' in locals() and hasattr(response, 'text') else 'N/A'}")
                break

            docs_page = data.get("ResultatenLijst", [])
            if not docs_page:
                print("No more document entries found in the list.")
                break 

            documents_list_meta.extend(docs_page)
            print(f"Page {page_num}: Fetched {len(docs_page)} document entries. Total metadata entries: {len(documents_list_meta)}")
            skip += top
            page_num += 1
            # time.sleep(0.1) # Verwijderd voor "full gass"

        if not documents_list_meta:
            print("No document metadata was fetched. Exiting.")
            return

        print(f"\nTotal document metadata entries found: {len(documents_list_meta)}")
        print("Now fetching full documents...")

        # Gebruik tqdm voor een progress bar 
        for doc_meta in tqdm(documents_list_meta, desc="Fetching full documents", unit="doc"):
            doc_id = doc_meta.get("Id")
            if not doc_id:
                log_error(f"Document metadata missing 'Id': {doc_meta}")
                continue
            
            doc_data = fetch_document(session, doc_id) # Geef session door
            if doc_data:
                file_path = os.path.join(OUTPUT_DIR, f"document_{doc_id}_raw.json")
                try:
                    with open(file_path, 'w', encoding='utf-8') as f:
                        json.dump(doc_data, f, indent=2, ensure_ascii=False)
                except IOError as e_io:
                    log_error(f"IOError writing document {doc_id} to {file_path}: {e_io}")
                except Exception as e_file:
                    log_error(f"Unexpected error writing document {doc_id} to {file_path}: {e_file}")
            # else: fetch_document logt zelf al fouten als het None retourneert

    else: # ID-range mode
        if start_id is None or num_docs is None or not isinstance(num_docs, int) or num_docs <= 0:
            msg = "Error: start_id and a positive integer num_docs must be provided when full_codex is False."
            print(msg)
            log_error(msg)
            return

        print(f"Fetching {num_docs} documents starting from ID {start_id}...")
        # Gebruik tqdm voor een progress bar
        for i in tqdm(range(num_docs), desc="Fetching documents by ID range", unit="doc"):
            current_id = start_id + i
            doc_data = fetch_document(session, current_id) # Geef session door
            if doc_data:
                file_path = os.path.join(OUTPUT_DIR, f"document_{current_id}_raw.json")
                try:
                    with open(file_path, 'w', encoding='utf-8') as f:
                        json.dump(doc_data, f, indent=2, ensure_ascii=False)
                except IOError as e_io:
                    log_error(f"IOError writing document {current_id} to {file_path}: {e_io}")
                except Exception as e_file:
                    log_error(f"Unexpected error writing document {current_id} to {file_path}: {e_file}")
            # else: fetch_document logt zelf al fouten

# Configuratie voor het ophalen van documenten
START_ID = 10001   # Pas aan indien nodig
NUM_DOCS = 40000   # Aantal documenten om op te halen als FULL_CODEX False is
FULL_CODEX = True  # Zet op True om de volledige codex te proberen, False voor een ID range

# Start het ophalen van documenten

get_documents(
    session=SESSION, # Geef het session object mee
    full_codex=FULL_CODEX,
    start_id=START_ID,
    num_docs=NUM_DOCS
)

print(f"Script finished. Check '{OUTPUT_DIR}' for documents and '{ERROR_LOG}' for any errors.")

# Missing ID + check html in Json

Het viel op dat er missing ID's waren. Bovendien waren er Json's die HTML bevatten. Moest je willen omzetten naar embeddings of markdown, dan zal je de data moeten cleanen. Dit is ook de reden waarom ik ervoor gekozen heb om de documenten te scrapen via HTML (zie later) op de website omdat dit minder werk was. Bovendien zijn er voor een of andere reden absoluut geen rate limits. 


In [ ]:
import os
import json
import re

# --- Configuratie ---
DATA_DIRECTORY = "codexjson"  # De map waar je gescrapete JSON-bestanden staan
REPORTS_DIRECTORY = "reports"

# Zorg ervoor dat de rapportenmap bestaat
os.makedirs(REPORTS_DIRECTORY, exist_ok=True)

# --- Functie om ontbrekende documentnummers te vinden ---
def find_missing_document_numbers(directory):
    """
    Vindt ontbrekende documentnummers in een gegeven map.
    Bestanden moeten het patroon 'document_{nummer}_raw.json' volgen.
    """
    files = [f for f in os.listdir(directory) if f.startswith("document_") and f.endswith("_raw.json")]

    numbers = []
    for file in files:
        match = re.search(r'document_(\d+)_raw\.json', file)
        if match:
            numbers.append(int(match.group(1)))

    if not numbers:
        return {
            "first_number": None,
            "last_number": None,
            "total_files_found": 0,
            "missing_numbers": [],
            "missing_count": 0,
            "directory_scanned": directory
        }

    numbers.sort()
    missing = []
    if numbers: # Alleen als er nummers zijn gevonden
        for i in range(numbers[0], numbers[-1] + 1):
            if i not in numbers:
                missing.append(i)

    return {
        "first_number": numbers[0] if numbers else None,
        "last_number": numbers[-1] if numbers else None,
        "total_files_found": len(numbers),
        "missing_numbers": missing,
        "missing_count": len(missing),
        "directory_scanned": directory
    }

def save_missing_report_to_json(data, report_dir):
    """
    Slaat het rapport over ontbrekende documenten op als JSON.
    """
    filepath = os.path.join(report_dir, "missing_document_numbers.json")
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"Rapport over ontbrekende documenten opgeslagen in: {filepath}")

# --- Functie om HTML in JSON-bestanden te detecteren ---
def find_documents_with_html(directory):
    """
    Detecteert JSON-bestanden in een map die HTML-tags lijken te bevatten.
    """
    files_with_html = []
    # Regex om te zoeken naar veelvoorkomende HTML-tags (case-insensitive)
    # Dit is een heuristiek en kan valse positieven/negatieven hebben.
    html_pattern = re.compile(r"<[^>]+>", re.IGNORECASE)

    json_files = [f for f in os.listdir(directory) if f.startswith("document_") and f.endswith("_raw.json")]

    for filename in json_files:
        filepath = os.path.join(directory, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content_str = f.read() # Lees het hele bestand als string
                
                # Simpele check op HTML-tags in de string-representatie
                if html_pattern.search(content_str):
                    # Probeer het document ID te extraheren voor een schonere lijst
                    match = re.search(r'document_(\d+)_raw\.json', filename)
                    if match:
                        files_with_html.append(int(match.group(1)))
                    else:
                        files_with_html.append(filename) # Fallback naar bestandsnaam
        except Exception as e:
            print(f"Fout bij het lezen of verwerken van bestand {filepath}: {e}")
            # Optioneel: log deze fout naar je error_log.txt
            # log_error(f"Fout bij HTML-check voor {filepath}: {e}")


    return {
        "documents_containing_html": sorted(list(set(files_with_html))), # Sorteer en maak uniek
        "count": len(set(files_with_html)),
        "directory_scanned": directory
    }

def save_html_report_to_json(data, report_dir):
    """
    Slaat het rapport over documenten met HTML op als JSON.
    """
    filepath = os.path.join(report_dir, "documents_with_html.json")
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"Rapport over documenten met HTML opgeslagen in: {filepath}")


# --- Voer de analyses uit en sla rapporten op ---

print("--- Analyse van Ontbrekende Documenten ---")
missing_docs_result = find_missing_document_numbers(DATA_DIRECTORY)
save_missing_report_to_json(missing_docs_result, REPORTS_DIRECTORY)

print(f"Directory gescand: {missing_docs_result['directory_scanned']}")
print(f"Eerste documentnummer gevonden: {missing_docs_result['first_number']}")
print(f"Laatste documentnummer gevonden: {missing_docs_result['last_number']}")
print(f"Totaal aantal bestanden gevonden: {missing_docs_result['total_files_found']}")
if missing_docs_result['missing_count'] > 0:
    print(f"Aantal ontbrekende documentnummers: {missing_docs_result['missing_count']}")
    # print(f"Ontbrekende nummers: {missing_docs_result['missing_numbers']}") # Kan lang zijn, optioneel
else:
    print("Geen ontbrekende documentnummers gevonden in de reeks.")

print("\n--- Analyse van HTML in Documenten ---")
html_docs_result = find_documents_with_html(DATA_DIRECTORY)
save_html_report_to_json(html_docs_result, REPORTS_DIRECTORY)

print(f"Directory gescand: {html_docs_result['directory_scanned']}")
print(f"Aantal documenten met vermoedelijke HTML: {html_docs_result['count']}")
if html_docs_result['count'] > 0:
    # print(f"Document ID's met HTML: {html_docs_result['documents_containing_html']}") # Kan lang zijn
    pass


print(f"\nAnalyses voltooid. Rapporten zijn te vinden in de map '{REPORTS_DIRECTORY}'.")


# Kabouter plot

Welke data kan ik hier vinden in mijn melkherberg? Deze cell maakt 4 plots op basis van de codexjson map die werd aangemaakt in de eerste cell. We keken naar het aantal documenten over de jaren heen en het aantal artikels. Er komt gemiddeld meer wetgeving bij. Zowel op document, als artikel niveau.

In [ ]:
import json
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

DATA_DIRECTORY = "codexjson"
HISTORICAL_START_DOC = 1980
HISTORICAL_END_DOC = 2024
FORECAST_START_DOC = 2025
FORECAST_END_DOC = 2030

# --- Helperfunctie voor het laden van jaartallen (documentniveau) ---
def load_document_years(directory, start_yr=None, end_yr=None):
    years_loaded = []
    file_list = glob.glob(os.path.join(directory, '*.json'))
    for file_path in file_list:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            if not data:
                continue
            document_object = data.get('Document')
            bs_date_value = None
            if isinstance(document_object, dict):
                bs_date_value = document_object.get('BSDatum')
            if bs_date_value:
                try:
                    year_str = str(bs_date_value).strip()[:4]
                    year = int(year_str)
                    if start_yr and year < start_yr:
                        continue
                    if end_yr and year > end_yr:
                        continue
                    years_loaded.append(year)
                except (ValueError, TypeError):
                    continue
        except Exception:
            continue
    return years_loaded

# --- Functie voor het plotten van publicatiedata (documentniveau) ---
def plot_document_publications(years_data, title_suffix):
    plt.figure(figsize=(12, 7)) # Nieuwe figuur voor elke plot
    ax = plt.gca() # Haal de current axes op
    sns.set_style("whitegrid")

    if not years_data:
        ax.text(0.5, 0.5, "Geen data gevonden", ha='center', va='center', fontsize=12)
        ax.set_title(f"Publicaties in Vlaamse Codex (documentniveau)\n{title_suffix}")
        plt.tight_layout()
        plt.show()
        return

    year_counts = pd.Series(years_data).value_counts().sort_index()
    if year_counts.empty:
        ax.text(0.5, 0.5, "Geen data na tellen", ha='center', va='center', fontsize=12)
        ax.set_title(f"Publicaties in Vlaamse Codex (documentniveau)\n{title_suffix}")
        plt.tight_layout()
        plt.show()
        return

    mean_val = year_counts.mean()
    sns.lineplot(x=year_counts.index, y=year_counts.values, marker='o', ax=ax, label='Documenten per jaar')
    x_numeric = year_counts.index.astype(float)
    y_numeric = year_counts.values.astype(float)

    if len(x_numeric) >= 2:
        sns.regplot(x=x_numeric, y=y_numeric, scatter=False, ax=ax, color='green', ci=None, line_kws={'label': 'Lineaire trend', 'linewidth': 1, 'alpha': 0.8})
    
    ax.axhline(y=mean_val, color='red', linestyle='--', linewidth=2.5, alpha=0.9, label=f'Gemiddeld ({mean_val:.1f} doc/jaar)')
    y_max_val = max(year_counts.max(), mean_val) if not year_counts.empty else mean_val
    ax.set_ylim(0, y_max_val * 1.15 if y_max_val > 0 else 10)
    ax.set_title(f"Publicaties in Vlaamse Codex (documentniveau)\n{title_suffix}", fontsize=14, pad=10)
    ax.set_xlabel('Jaar', fontsize=12)
    ax.set_ylabel('Aantal documenten', fontsize=12)
    if not year_counts.empty:
        tick_years = year_counts.index.unique().astype(int)
        ax.set_xticks(tick_years)
        ax.tick_params(axis='x', rotation=45)
    ax.legend()
    plt.tight_layout()
    plt.show()

# --- Functie voor plot 3: Documenten historisch & prognose ---
def plot_document_forecast(years_hist_data):
    plt.figure(figsize=(12, 7)) # Nieuwe figuur
    ax = plt.gca()
    sns.set_style("whitegrid")

    title_str = f"Publicaties Vlaamse Codex (documentniveau)\nHistorisch {HISTORICAL_START_DOC}-{HISTORICAL_END_DOC} & Prognose {FORECAST_START_DOC}-{FORECAST_END_DOC}"

    if not years_hist_data:
        ax.text(0.5, 0.5, "Geen historische data", ha='center', va='center', fontsize=12)
        ax.set_title(title_str)
        plt.tight_layout()
        plt.show()
        return

    year_counts_hist = pd.Series(years_hist_data).value_counts().sort_index()
    if year_counts_hist.empty or len(year_counts_hist.index) < 2:
        ax.text(0.5, 0.5, "Te weinig data voor prognose", ha='center', va='center', fontsize=12)
        ax.set_title(title_str)
        plt.tight_layout()
        plt.show()
        return

    x_hist = year_counts_hist.index.astype(float)
    y_hist = year_counts_hist.values.astype(float)
    slope, intercept = np.polyfit(x_hist, y_hist, 1)
    
    full_years_range = np.arange(int(x_hist.min()), FORECAST_END_DOC + 1)
    predicted_counts_full = slope * full_years_range + intercept
    predicted_counts_full[predicted_counts_full < 0] = 0
    
    future_years_range = np.arange(FORECAST_START_DOC, FORECAST_END_DOC + 1)
    forecast_counts_values = slope * future_years_range + intercept
    forecast_counts_values[forecast_counts_values < 0] = 0
    mean_hist = year_counts_hist.mean()

    sns.lineplot(x=year_counts_hist.index, y=year_counts_hist.values, marker='o', ax=ax, label='Documenten per jaar (historisch)')
    ax.plot(full_years_range, predicted_counts_full, color='green', linewidth=1, alpha=0.8, label='Lineaire trend & prognose')
    
    plot_future_years = future_years_range[future_years_range >= FORECAST_START_DOC]
    plot_future_counts = forecast_counts_values[future_years_range >= FORECAST_START_DOC]
    if plot_future_years.size > 0:
        ax.plot(plot_future_years, plot_future_counts, color='darkorange', linestyle='--', linewidth=2, label=f'Prognose ({plot_future_years.min()}-{plot_future_years.max()})')

    ax.axhline(y=mean_hist, color='red', linestyle='--', linewidth=2.5, alpha=0.9, label=f'Gemiddeld historisch ({mean_hist:.1f} doc/jaar)')
    y_max_plot_fc = max(year_counts_hist.max() if not year_counts_hist.empty else 0, mean_hist if mean_hist else 0)
    if predicted_counts_full.size > 0: y_max_plot_fc = max(y_max_plot_fc, predicted_counts_full.max())
    ax.set_ylim(0, y_max_plot_fc * 1.15 if y_max_plot_fc > 0 else 10)
    ax.set_title(title_str, fontsize=14, pad=10)
    ax.set_xlabel('Jaar', fontsize=12)
    ax.set_ylabel('Aantal documenten', fontsize=12)
    
    combined_ticks_fc = np.unique(np.concatenate((year_counts_hist.index.astype(int), future_years_range[future_years_range >= year_counts_hist.index.min()]))).astype(int)
    ax.set_xticks(combined_ticks_fc)
    ax.tick_params(axis='x', rotation=45)
    ax.legend()
    plt.tight_layout()
    plt.show()

# --- Functies voor plot 4: Artikelniveau ---
def count_articles_in_document(data):
    if "Inhoud" in data and data["Inhoud"] and \
       "Artikelen" in data["Inhoud"] and data["Inhoud"]["Artikelen"] is not None:
        if isinstance(data["Inhoud"]["Artikelen"], list):
            return len(data["Inhoud"]["Artikelen"])
        else:
            return 0 
    count = 0
    if "Inhoudstafel" in data and data["Inhoudstafel"] and \
       "Items" in data["Inhoudstafel"] and isinstance(data["Inhoudstafel"]["Items"], list):
        for item in data["Inhoudstafel"]["Items"]:
            if isinstance(item, dict) and item.get("ArtikelType") == "ART.":
                count += 1
    return count

def plot_artikels_per_jaar_with_forecast(directory=DATA_DIRECTORY, HISTORICAL_START=1990, HISTORICAL_END=2024, FORECAST_START=2025, FORECAST_END=2030):
    plt.figure(figsize=(12, 7)) # Nieuwe figuur
    ax = plt.gca()
    sns.set_style("whitegrid")
    
    title_str = f'Aantal Artikels per Jaar\n(historisch: {HISTORICAL_START}-{HISTORICAL_END}, prognose: {FORECAST_START}-{FORECAST_END})'
    articles_by_year = {}

    for file_path in glob.glob(os.path.join(directory, "*.json")):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            document = data.get("Document", {})
            bs_datum = document.get("BSDatum")
            if not bs_datum:
                continue
            year = int(bs_datum[:4])
            if HISTORICAL_START <= year <= HISTORICAL_END:
                count = count_articles_in_document(data)
                articles_by_year[year] = articles_by_year.get(year, 0) + count
        except Exception:
            continue
    
    if not articles_by_year:
        ax.text(0.5, 0.5, f"Geen artikeldata {HISTORICAL_START}-{HISTORICAL_END}", ha='center', va='center', fontsize=12)
        ax.set_title(title_str)
        plt.tight_layout()
        plt.show()
        return

    series = pd.Series(articles_by_year).sort_index()
    if series.empty or len(series.index) < 2:
        ax.text(0.5, 0.5, "Te weinig artikeldata voor prognose", ha='center', va='center', fontsize=12)
        ax.set_title(title_str)
        plt.tight_layout()
        plt.show()
        return

    mean_count = series.mean()
    x = np.array(series.index)
    y = np.array(series.values)
    slope, intercept = np.polyfit(x, y, 1)
    
    full_years = np.arange(int(x.min()), FORECAST_END + 1) 
    predicted_counts = slope * full_years + intercept
    predicted_counts[predicted_counts < 0] = 0
    
    future_years = np.arange(FORECAST_START, FORECAST_END + 1)
    forecast_counts = slope * future_years + intercept
    forecast_counts[forecast_counts < 0] = 0

    sns.lineplot(x=series.index, y=series.values, marker='o', ax=ax, label='Artikels per jaar (historisch)')
    ax.plot(full_years, predicted_counts, color='green', linewidth=1, alpha=0.8, label='Lineaire trend en prognose')
    
    plot_future_years_art = future_years[future_years >= FORECAST_START]
    plot_future_counts_art = forecast_counts[future_years >= FORECAST_START]

    if plot_future_years_art.size > 0:
        ax.plot(plot_future_years_art, plot_future_counts_art, color='darkorange', linestyle='--', linewidth=2, label=f'Prognose ({plot_future_years_art.min()}-{plot_future_years_art.max()})')
    
    ax.axhline(y=mean_count, color='red', linestyle='--', linewidth=2.5, alpha=0.9, label=f'Gemiddeld ({mean_count:.1f} artikels/jaar)')
    y_max = max(series.max() if not series.empty else 0, mean_count if mean_count else 0)
    if predicted_counts.size > 0 : y_max = max(y_max, predicted_counts.max())
    ax.set_ylim(0, y_max * 1.15 if y_max > 0 else 10)
    ax.set_title(title_str, fontsize=14, pad=10)
    ax.set_xlabel('Jaar', fontsize=12)
    ax.set_ylabel('Totaal aantal artikels', fontsize=12)
    
    combined_ticks_art = np.unique(np.concatenate((series.index.astype(int), future_years[future_years >= series.index.min()]))).astype(int)
    ax.set_xticks(combined_ticks_art)
    ax.tick_params(axis='x', rotation=45)
    ax.legend()
    plt.tight_layout()
    plt.show()

# --- Genereer de 4 plots ---

# Plot 1
print("--- Plot 1: Documenten 1980 - 2024 ---")
years_plot1 = load_document_years(DATA_DIRECTORY, start_yr=1980, end_yr=2024)
plot_document_publications(years_plot1, "1980 - 2024")

# Plot 2
print("\n--- Plot 2: Documenten 2000 - 2024 ---")
years_plot2 = load_document_years(DATA_DIRECTORY, start_yr=2000, end_yr=2024)
plot_document_publications(years_plot2, "2000 - 2024")

# Plot 3
print(f"\n--- Plot 3: Documenten Historisch {HISTORICAL_START_DOC}-{HISTORICAL_END_DOC} & Prognose {FORECAST_START_DOC}-{FORECAST_END_DOC} ---")
years_hist_plot3 = load_document_years(DATA_DIRECTORY, start_yr=HISTORICAL_START_DOC, end_yr=HISTORICAL_END_DOC)
plot_document_forecast(years_hist_plot3)

# Plot 4
HISTORICAL_START_ART = 1990 
HISTORICAL_END_ART = 2024
FORECAST_START_ART = 2025
FORECAST_END_ART = 2030
print(f"\n--- Plot 4: Artikelen Historisch {HISTORICAL_START_ART}-{HISTORICAL_END_ART} & Prognose {FORECAST_START_ART}-{FORECAST_END_ART} ---")
plot_artikels_per_jaar_with_forecast(directory=DATA_DIRECTORY, 
                                     HISTORICAL_START=HISTORICAL_START_ART, 
                                     HISTORICAL_END=HISTORICAL_END_ART, 
                                     FORECAST_START=FORECAST_START_ART, 
                                     FORECAST_END=FORECAST_END_ART)

# --- Print voorspellingsdetails (optioneel) ---

# Documenten
print(f"\n--- Voorspellingsdetails Documenten (Historisch: {HISTORICAL_START_DOC}-{HISTORICAL_END_DOC}, Prognose: {FORECAST_START_DOC}-{FORECAST_END_DOC}) ---")
if years_hist_plot3: # Hergebruik de data geladen voor plot 3
    year_counts_hist_doc = pd.Series(years_hist_plot3).value_counts().sort_index()
    if not year_counts_hist_doc.empty and len(year_counts_hist_doc.index) >= 2:
        x_hist_doc = year_counts_hist_doc.index.astype(float)
        y_hist_doc = year_counts_hist_doc.values.astype(float)
        slope_doc, intercept_doc = np.polyfit(x_hist_doc, y_hist_doc, 1)
        future_years_doc_range = np.arange(FORECAST_START_DOC, FORECAST_END_DOC + 1)
        forecast_counts_doc_values = slope_doc * future_years_doc_range + intercept_doc
        forecast_counts_doc_values[forecast_counts_doc_values < 0] = 0
        for year_fc, count_fc in zip(future_years_doc_range, forecast_counts_doc_values):
            print(f"Jaar {year_fc} (documenten): Voorspeld aantal = {count_fc:.1f}")
    else:
        print("Niet genoeg data voor documenten voorspelling.")
else:
    print("Geen historische data geladen voor documenten voorspelling (plot 3).")

# Artikelen
print(f"\n--- Voorspellingsdetails Artikelen (Historisch: {HISTORICAL_START_ART}-{HISTORICAL_END_ART}, Prognose: {FORECAST_START_ART}-{FORECAST_END_ART}) ---")
articles_by_year_print = {}
# Herlaad data specifiek voor de historische periode van artikelen
for file_path in glob.glob(os.path.join(DATA_DIRECTORY, "*.json")):
    try:
        with open(file_path, "r", encoding="utf-8") as f: data = json.load(f)
        document = data.get("Document", {}); bs_datum = document.get("BSDatum")
        if not bs_datum: continue
        year = int(bs_datum[:4])
        if HISTORICAL_START_ART <= year <= HISTORICAL_END_ART: # Gebruik HISTORICAL_START_ART en HISTORICAL_END_ART
            articles_by_year_print[year] = articles_by_year_print.get(year, 0) + count_articles_in_document(data)
    except Exception: continue

if articles_by_year_print:
    series_art_print = pd.Series(articles_by_year_print).sort_index()
    if not series_art_print.empty and len(series_art_print.index) >=2:
        x_art = np.array(series_art_print.index); y_art = np.array(series_art_print.values)
        slope_art, intercept_art = np.polyfit(x_art, y_art, 1)
        future_years_art_range = np.arange(FORECAST_START_ART, FORECAST_END_ART + 1)
        forecast_counts_art_values = slope_art * future_years_art_range + intercept_art
        forecast_counts_art_values[forecast_counts_art_values < 0] = 0
        for year_fc, count_fc in zip(future_years_art_range, forecast_counts_art_values):
            print(f"Jaar {year_fc} (artikelen): Voorspeld aantal = {count_fc:.1f}")
    else:
        print("Niet genoeg data voor artikelen voorspelling.")
else:
    print("Geen historische data voor artikelen voorspelling.")


# Sorteren van de Vlaamse Codex

Het organiseren van de verschillende wetgevingsdocumenten is makkelijk. We hebben ‘decreet’ verder opgesplitst voor 'decreet' in heeft inhoud = true en false. Momenteel zijn er 2359 decreten die geldig zijn. Deze zijn te vinden in de **map O_type_decreet** 

In [ ]:
import json
import os
import shutil
import re

# --- Configuratie ---
# Bronmap met de originele JSON-bestanden
source_dir = 'codexjson'
# Doelmap waar de georganiseerde bestanden komen
target_base_dir = 'organized_documents_v3' # Nieuwe naam om conflicten te vermijden

# Speciale behandeling voor het type "Decreet"
DECREET_TYPE_NAME = "Decreet"
DECREET_FOLDER_PREFIX = "0_Type_" # Prefix om "Decreet" bovenaan te sorteren

# --- Stap 1: Organiseer alle documenten op basis van WetgevingDocumentType ---

print(f"--- Start Stap 1: Organiseren op Documenttype ---")
os.makedirs(target_base_dir, exist_ok=True)
print(f"Hoofddoelmap '{target_base_dir}' is gereed.")

for filename in os.listdir(source_dir):
    if filename.endswith('.json'):
        source_file_path = os.path.join(source_dir, filename)
        
        try:
            with open(source_file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            document_type_value = None
            if 'Document' in data and isinstance(data.get('Document'), dict):
                document_type_value = data['Document'].get('WetgevingDocumentType')

            if isinstance(document_type_value, str) and document_type_value.strip():
                folder_name_from_type = document_type_value.strip()
            else:
                folder_name_from_type = "Onbekend_Type"
                # print(f"Info (Stap 1): Bestand {filename} - 'WetgevingDocumentType' onbekend. Gebruikt 'Onbekend_Type'.")

            sanitized_folder_name_base = re.sub(r'[\\/*?:"<>|]', '_', folder_name_from_type)
            
            if folder_name_from_type == DECREET_TYPE_NAME:
                final_folder_name = f"{DECREET_FOLDER_PREFIX}{sanitized_folder_name_base}"
            else:
                final_folder_name = sanitized_folder_name_base # Geen prefix voor andere types

            target_subdir = os.path.join(target_base_dir, final_folder_name)
            os.makedirs(target_subdir, exist_ok=True)
            
            target_file_path = os.path.join(target_subdir, filename)
            shutil.copy2(source_file_path, target_file_path)
            # print(f"Gekopieerd (Stap 1): '{source_file_path}' naar '{target_file_path}'")
            
        except json.JSONDecodeError:
            print(f"Fout (Stap 1): Kon JSON niet decoderen uit {source_file_path}. Bestand overgeslagen.")
        except Exception as e:
            print(f"Fout (Stap 1): Een onverwachte fout trad op bij het verwerken van {source_file_path}: {e}. Bestand overgeslagen.")

print(f"\n--- Stap 1 Voltooid: Bestandsorganisatie op type in '{target_base_dir}' ---")

# --- Stap 2: Splits de "Decreet" map verder op basis van 'HeeftInhoud' ---
# We zullen verder werken met de inhoud van de map "0_Type_Decreet", 
# met name de submappen 'HeeftInhoud_True' en 'HeeftInhoud_False'.

print(f"\n--- Start Stap 2: Opsplitsen van '{DECREET_TYPE_NAME}' map ---")

sanitized_decreet_base_name = re.sub(r'[\\/*?:"<>|]', '_', DECREET_TYPE_NAME)
decreet_main_folder_path = os.path.join(target_base_dir, f"{DECREET_FOLDER_PREFIX}{sanitized_decreet_base_name}")

if not os.path.isdir(decreet_main_folder_path):
    print(f"Info (Stap 2): De map '{decreet_main_folder_path}' bestaat niet. Mogelijk waren er geen documenten van het type '{DECREET_TYPE_NAME}'. Stap 2 overgeslagen.")
else:
    true_dir = os.path.join(decreet_main_folder_path, 'HeeftInhoud_True')
    false_dir = os.path.join(decreet_main_folder_path, 'HeeftInhoud_False')

    os.makedirs(true_dir, exist_ok=True)
    os.makedirs(false_dir, exist_ok=True)
    print(f"Doelmappen voor 'HeeftInhoud' in '{decreet_main_folder_path}' zijn aangemaakt/geverifieerd.")

    files_to_move = []
    for item_name in os.listdir(decreet_main_folder_path):
        source_item_path = os.path.join(decreet_main_folder_path, item_name)
        if os.path.isfile(source_item_path) and item_name.endswith('.json'):
            files_to_move.append((item_name, source_item_path))

    for item_name, source_item_path in files_to_move:
        try:
            with open(source_item_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            heeft_inhoud_value = None
            document_data = data.get('Document')
            moved = False

            if isinstance(document_data, dict):
                heeft_inhoud_value = document_data.get('HeeftInhoud')

            if isinstance(heeft_inhoud_value, bool):
                if heeft_inhoud_value: # True
                    target_file_path = os.path.join(true_dir, item_name)
                    shutil.move(source_item_path, target_file_path)
                    # print(f"Verplaatst (Stap 2): '{item_name}' naar '{os.path.basename(true_dir)}'")
                    moved = True
                else: # False
                    target_file_path = os.path.join(false_dir, item_name)
                    shutil.move(source_item_path, target_file_path)
                    # print(f"Verplaatst (Stap 2): '{item_name}' naar '{os.path.basename(false_dir)}'")
                    moved = True
            
            if not moved:
                # Als het bestand niet verplaatst is (omdat HeeftInhoud geen boolean was of ontbrak),
                # blijft het in de '0_Type_Decreet' map staan.
                print(f"Info (Stap 2): Bestand {item_name} - 'HeeftInhoud' is geen boolean (waarde: '{heeft_inhoud_value}') of 'Document' structuur is onverwacht. Bestand blijft in '{decreet_main_folder_path}'.")

        except json.JSONDecodeError:
            print(f"Fout (Stap 2): Kon JSON niet decoderen uit {source_item_path}. Bestand blijft in '{decreet_main_folder_path}'.")
        except Exception as e:
            print(f"Fout (Stap 2): Onverwachte fout bij {source_item_path}: {e}. Bestand blijft in '{decreet_main_folder_path}'.")
            
    print(f"\n--- Stap 2 Voltooid: Herschikken van '{DECREET_TYPE_NAME}' map ---")
    print(f"De '{DECREET_TYPE_NAME}' bestanden zijn nu, indien mogelijk, onderverdeeld in '{os.path.basename(true_dir)}' en '{os.path.basename(false_dir)}' binnen '{decreet_main_folder_path}'.")
    print(f"Bestanden met onduidelijke 'HeeftInhoud' status blijven in de hoofmap '{decreet_main_folder_path}'.")

print("\nAlle bestandsorganisatie voltooid.")

# HTML scrapen van alle decreten met heeftinhoud_true

Scraper die alle geldige decreten via de website van de Vlaamse Codex ophaalt. Dit gebeurt per geldig decreet, voor zowel de versie met als zonder annotaties.


In [ ]:
import os
import re
import asyncio
import aiohttp 
import nest_asyncio 
import time 

nest_asyncio.apply() 

# --- Configuratie ---
# Pad naar de map met JSON-bestanden van decreten die inhoud hebben
SOURCE_JSON_DIR = os.path.join('organized_documents_v3', '0_Type_Decreet', 'HeeftInhoud_True')

# Hoofddoelmap voor de gedownloade HTML-bestanden
TARGET_HTML_BASE_DIR = os.path.join('organized_documents_v3', '0_Type_Decreet', 'HTML_Bestanden')
TARGET_HTML_WITH_ANNOTATIONS_DIR = os.path.join(TARGET_HTML_BASE_DIR, 'Met_Annotaties')
TARGET_HTML_WITHOUT_ANNOTATIONS_DIR = os.path.join(TARGET_HTML_BASE_DIR, 'Zonder_Annotaties')

# Basis URL voor het downloaden van de HTML-documenten
BASE_DOWNLOAD_URL = "https://codex.vlaanderen.be/PrintDocument.ashx"

# Parameters voor asynchrone downloads
CONCURRENT_DOWNLOADS = 25
REQUEST_TIMEOUT = 60
RETRY_ATTEMPTS = 2
RETRY_DELAY = 5

# --- Helperfuncties ---

def extract_id_from_filename(filename):
    """Extraheert het document-ID uit de bestandsnaam (bv. document_12345_raw.json -> 12345)."""
    match = re.match(r"document_(\d+)_raw\.json", filename)
    if match:
        return match.group(1)
    return None

async def fetch_html_with_retries(session, doc_id, geannoteerd_value, filename_for_log):
    """Downloadt één HTML-versie asynchroon met nieuwe pogingen."""
    params = {
        'id': doc_id,
        'datum': '',
        'geannoteerd': str(geannoteerd_value).lower(),
        'print': 'false'
    }
    # Definieer een timeout object voor de request
    timeout = aiohttp.ClientTimeout(total=REQUEST_TIMEOUT, connect=20) # bv. 20s voor connect, rest voor read

    for attempt in range(RETRY_ATTEMPTS + 1):
        try:
            async with session.get(BASE_DOWNLOAD_URL, params=params, timeout=timeout) as response:
                response.raise_for_status()
                return await response.text()
        except asyncio.TimeoutError:
            print(f"Timeout (poging {attempt + 1}/{RETRY_ATTEMPTS + 1}) ID {doc_id} (annot={geannoteerd_value}) ({filename_for_log})")
            if attempt < RETRY_ATTEMPTS:
                await asyncio.sleep(RETRY_DELAY)
            else:
                return None
        except aiohttp.ClientResponseError as http_err:
            print(f"HTTP fout {http_err.status} (poging {attempt + 1}/{RETRY_ATTEMPTS + 1}) ID {doc_id} (annot={geannoteerd_value}) ({filename_for_log}): {http_err.message}")
            if attempt < RETRY_ATTEMPTS and http_err.status != 404: # Niet opnieuw proberen bij 404 (Not Found)
                await asyncio.sleep(RETRY_DELAY)
            else:
                return None
        except aiohttp.ClientConnectionError as conn_err:
            print(f"Connectie fout (poging {attempt + 1}/{RETRY_ATTEMPTS + 1}) ID {doc_id} (annot={geannoteerd_value}) ({filename_for_log}): {conn_err}")
            if attempt < RETRY_ATTEMPTS:
                await asyncio.sleep(RETRY_DELAY)
            else:
                return None
        except Exception as e:
            print(f"Algemene fout (poging {attempt + 1}/{RETRY_ATTEMPTS + 1}) ID {doc_id} (annot={geannoteerd_value}) ({filename_for_log}): {e}")
            if attempt < RETRY_ATTEMPTS:
                await asyncio.sleep(RETRY_DELAY)
            else:
                return None
    return None

async def process_single_document(session, doc_id, geannoteerd_value, original_filename):
    """Verwerkt het downloaden en opslaan voor één document en één annotatie-instelling."""
    html_content = await fetch_html_with_retries(session, doc_id, geannoteerd_value, original_filename)
    if html_content:
        annot_suffix = "met_annotaties" if geannoteerd_value else "zonder_annotaties"
        html_filename = f"document_{doc_id}_{annot_suffix}.html"
        
        target_dir = TARGET_HTML_WITH_ANNOTATIONS_DIR if geannoteerd_value else TARGET_HTML_WITHOUT_ANNOTATIONS_DIR
        html_file_path = os.path.join(target_dir, html_filename)
            
        try:
            with open(html_file_path, 'w', encoding='utf-8') as f_html:
                f_html.write(html_content)
            # print(f"Opgeslagen: '{html_file_path}' (ID: {doc_id}, Annotaties: {geannoteerd_value})") 
        except IOError as io_err:
            print(f"IO Fout bij opslaan {html_file_path}: {io_err}")
        except Exception as e:
            print(f"Onverwachte fout bij opslaan {html_file_path}: {e}")

async def download_and_save_html_versions():
    """
    Doorloopt JSON-bestanden, extraheert ID's, downloadt asynchroon twee HTML-versies
    (met en zonder annotaties) en slaat deze op.
    """
    if not os.path.exists(SOURCE_JSON_DIR):
        print(f"Fout: De bronmap '{SOURCE_JSON_DIR}' met JSON-bestanden is niet gevonden.")
        return

    os.makedirs(TARGET_HTML_WITH_ANNOTATIONS_DIR, exist_ok=True)
    os.makedirs(TARGET_HTML_WITHOUT_ANNOTATIONS_DIR, exist_ok=True)
    print(f"Doelmappen in '{TARGET_HTML_BASE_DIR}' zijn aangemaakt/geverifieerd.")
    print(f"Starten met het downloaden van HTML-versies voor documenten in '{SOURCE_JSON_DIR}'...")

    doc_ids_to_process = []
    for filename in os.listdir(SOURCE_JSON_DIR):
        if filename.endswith(".json"):
            doc_id = extract_id_from_filename(filename)
            if not doc_id:
                print(f"Waarschuwing: Kon geen ID extraheren uit bestandsnaam '{filename}'. Overgeslagen.")
                continue
            doc_ids_to_process.append((doc_id, filename))

    if not doc_ids_to_process:
        print("Geen JSON-bestanden gevonden om te verwerken.")
        return

    # ssl=False kan soms helpen met SSL-certificaat problemen.
    # force_close=True kan helpen om verbindingen sneller te sluiten na gebruik.
    connector = aiohttp.TCPConnector(limit_per_host=CONCURRENT_DOWNLOADS, ssl=False, force_close=True)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        for i, (doc_id, original_filename) in enumerate(doc_ids_to_process):
            tasks.append(asyncio.create_task(
                process_single_document(session, doc_id, True, original_filename)
            ))
            tasks.append(asyncio.create_task(
                process_single_document(session, doc_id, False, original_filename)
            ))
            
            # Beheer batchgrootte om niet te veel taken tegelijk in geheugen te hebben en de server niet te overspoelen
            if len(tasks) >= CONCURRENT_DOWNLOADS * 2 : # Wacht op een batch (aangezien we 2 taken per ID maken)
                print(f"Batch {i // CONCURRENT_DOWNLOADS + 1} gestart, wachten op voltooiing...")
                await asyncio.gather(*tasks)
                tasks = [] # Reset takenlijst voor volgende batch
                print(f"Batch voltooid, kleine pauze...")
                await asyncio.sleep(0.5) # Iets langere pauze tussen batches
        
        if tasks: # Verwerk resterende taken
            print("Verwerken van resterende taken...")
            await asyncio.gather(*tasks)

    print("\nDownloaden van HTML-versies voltooid.")

# --- Hoofduitvoering aangepast voor Jupyter Notebook ---
async def main_async_download_script(): 
    start_time = time.time()
    await download_and_save_html_versions()
    end_time = time.time()
    print(f"Totale downloadtijd: {end_time - start_time:.2f} seconden.")

# Roep de hoofdfunctie aan. Dankzij nest_asyncio.apply() bovenaan,
# zou asyncio.run() nu moeten werken in Jupyter.
if __name__ == '__main__': # Deze check is nuttig als je het script ook buiten Jupyter wilt runnen
    asyncio.run(main_async_download_script())
else: # Voor directe uitvoering in Jupyter cel
    asyncio.run(main_async_download_script())


# HTML naar markdown

Deze cell converteert automatisch alle HTML-bestanden uit de mappen "Met_Annotaties" en "Zonder_Annotaties" naar Markdown-bestanden met behulp MarkItDown. Door gebruik te maken van parallelle verwerking worden bestanden tegelijk omgezet, wat het proces aanzienlijk versnelt. De geconverteerde Markdown-bestanden worden overzichtelijk opgeslagen in aparte doelmappen per type annotatie.

In [ ]:
import os
import re
import subprocess
import shutil
import concurrent.futures
import time

# Definieer de bronmappen met de HTML-bestanden
SOURCE_HTML_DIRS = {
    'met_annotaties': os.path.join('organized_documents_v3', '0_Type_Decreet', 'HTML_Bestanden', 'Met_Annotaties'),
    'zonder_annotaties': os.path.join('organized_documents_v3', '0_Type_Decreet', 'HTML_Bestanden', 'Zonder_Annotaties')
}

# Definieer de doelmappen voor de geconverteerde Markdown-bestanden
TARGET_MD_DIRS = {
    'met_annotaties': os.path.join('organized_documents_v3', '0_Type_Decreet', 'Markdown_Bestanden', 'Met_Annotaties'),
    'zonder_annotaties': os.path.join('organized_documents_v3', '0_Type_Decreet', 'Markdown_Bestanden', 'Zonder_Annotaties')
}

# PERFORMANCE INSTELLINGEN
MAX_WORKERS = 8  # Aantal parallelle processen 
BATCH_SIZE = 50  # Aantal bestanden per batch voor voortgangsrapportage

# MarkItDown executable configuratie
MARKITDOWN_EXECUTABLE = "markitdown"

# Maak de doelmappen aan als deze nog niet bestaan
for target_dir in TARGET_MD_DIRS.values():
    os.makedirs(target_dir, exist_ok=True)
    print(f"Doelmap '{target_dir}' is aangemaakt/geverifieerd.")

def extract_id_from_html_filename(filename):
    """Extraheert het document-ID uit de HTML-bestandsnaam."""
    match = re.match(r"document_(\d+)_(met|zonder)_annotaties\.html", filename)
    if match:
        return match.group(1), match.group(2)
    return None, None

def process_single_file(file_info):
    """
    Verwerkt een enkel HTML-bestand naar Markdown.
    Deze functie wordt parallel uitgevoerd.
    """
    source_html_path, target_md_path, filename, doc_id = file_info
    
    # Skip als output bestand al bestaat en niet leeg is
    if os.path.exists(target_md_path) and os.path.getsize(target_md_path) > 0:
        return {
            'filename': filename,
            'doc_id': doc_id,
            'status': 'skipped',
            'message': 'Bestand bestaat al',
            'size': os.path.getsize(target_md_path)
        }
    
    # Stel het commando samen
    if isinstance(MARKITDOWN_EXECUTABLE, list):
        command = [*MARKITDOWN_EXECUTABLE, source_html_path, "-o", target_md_path]
    else:
        command = [MARKITDOWN_EXECUTABLE, source_html_path, "-o", target_md_path]
    
    try:
        # Voer markitdown uit met timeout om hangende processen te voorkomen
        process = subprocess.run(
            command, 
            capture_output=True, 
            text=True, 
            encoding='utf-8', 
            check=False,
            timeout=60  # 60 seconden timeout per bestand
        )
        
        if process.returncode == 0:
            if os.path.exists(target_md_path) and os.path.getsize(target_md_path) > 0:
                file_size = os.path.getsize(target_md_path)
                return {
                    'filename': filename,
                    'doc_id': doc_id,
                    'status': 'success',
                    'message': f'Succesvol geconverteerd ({file_size} bytes)',
                    'size': file_size
                }
            else:
                return {
                    'filename': filename,
                    'doc_id': doc_id,
                    'status': 'error',
                    'message': 'Output bestand leeg of niet aangemaakt',
                    'stdout': process.stdout[:200] if process.stdout else ''
                }
        else:
            return {
                'filename': filename,
                'doc_id': doc_id,
                'status': 'error',
                'message': f'Return code: {process.returncode}',
                'stderr': process.stderr[:200] if process.stderr else '',
                'stdout': process.stdout[:200] if process.stdout else ''
            }
    
    except subprocess.TimeoutExpired:
        return {
            'filename': filename,
            'doc_id': doc_id,
            'status': 'error',
            'message': 'Timeout na 60 seconden'
        }
    except Exception as e:
        return {
            'filename': filename,
            'doc_id': doc_id,
            'status': 'error',
            'message': f'Onverwachte fout: {str(e)}'
        }

def prepare_file_list():
    """Bereidt een lijst voor van alle te verwerken bestanden."""
    all_files = []
    
    for folder_type, source_dir in SOURCE_HTML_DIRS.items():
        if not os.path.exists(source_dir):
            print(f"Waarschuwing: De bronmap '{source_dir}' is niet gevonden. Overgeslagen.")
            continue
        
        target_dir = TARGET_MD_DIRS[folder_type]
        
        for filename in os.listdir(source_dir):
            if filename.endswith(".html"):
                doc_id, annotation_type = extract_id_from_html_filename(filename)
                
                if not doc_id:
                    print(f"Waarschuwing: Kon geen ID extraheren uit '{filename}'. Overgeslagen.")
                    continue
                
                source_html_path = os.path.join(source_dir, filename)
                md_filename = f"document_{doc_id}_{annotation_type}_annotaties_converted.md"
                target_md_path = os.path.join(target_dir, md_filename)
                
                all_files.append((source_html_path, target_md_path, filename, doc_id))
    
    return all_files

def convert_html_to_markdown_parallel():
    """
    Parallelle versie van de HTML naar Markdown conversie.
    """
    # Controleer of markitdown beschikbaar is
    executable_to_check = MARKITDOWN_EXECUTABLE if isinstance(MARKITDOWN_EXECUTABLE, str) else MARKITDOWN_EXECUTABLE[0]
    if not shutil.which(executable_to_check):
        print(f"FOUT: Het commando '{executable_to_check}' kon niet worden gevonden.")
        print(f"Zorg ervoor dat 'markitdown' correct is geïnstalleerd.")
        return
    
    # Bereid lijst van bestanden voor
    print("Voorbereiden van bestandslijst...")
    all_files = prepare_file_list()
    
    if not all_files:
        print("Geen bestanden gevonden om te verwerken.")
        return
    
    print(f"Gevonden {len(all_files)} bestanden om te verwerken.")
    print(f"Gebruik {MAX_WORKERS} parallelle workers.")
    
    # Statistieken
    total_converted = 0
    total_errors = 0
    total_skipped = 0
    total_size = 0
    
    start_time = time.time()
    
    # Parallelle verwerking met ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Submit alle taken
        future_to_file = {executor.submit(process_single_file, file_info): file_info for file_info in all_files}
        
        # Verwerk resultaten zodra ze beschikbaar komen
        for i, future in enumerate(concurrent.futures.as_completed(future_to_file)):
            result = future.result()
            
            # Update statistieken
            if result['status'] == 'success':
                total_converted += 1
                total_size += result.get('size', 0)
                print(f"✓ {result['filename']} - {result['message']}")
            elif result['status'] == 'skipped':
                total_skipped += 1
                total_size += result.get('size', 0)
                print(f"⏭ {result['filename']} - {result['message']}")
            else:
                total_errors += 1
                print(f"✗ {result['filename']} - {result['message']}")
                if 'stderr' in result and result['stderr']:
                    print(f"    Stderr: {result['stderr']}")
            
            # Voortgangsrapportage per batch
            if (i + 1) % BATCH_SIZE == 0:
                elapsed = time.time() - start_time
                rate = (i + 1) / elapsed
                eta = (len(all_files) - i - 1) / rate if rate > 0 else 0
                print(f"\n📊 Voortgang: {i + 1}/{len(all_files)} ({(i + 1)/len(all_files)*100:.1f}%)")
                print(f"⏱️  Snelheid: {rate:.1f} bestanden/sec, ETA: {eta:.0f} seconden\n")
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # Eindrapport
    print(f"\n{'='*80}")
    print(f"🎉 CONVERSIE VOLTOOID!")
    print(f"{'='*80}")
    print(f"📈 STATISTIEKEN:")
    print(f"   ✅ Succesvol geconverteerd: {total_converted}")
    print(f"   ⏭️  Overgeslagen (bestond al): {total_skipped}")
    print(f"   ❌ Fouten: {total_errors}")
    print(f"   📁 Totaal verwerkt: {total_converted + total_errors + total_skipped}")
    print(f"   💾 Totale output grootte: {total_size / (1024*1024):.1f} MB")
    print(f"")
    print(f"⏱️  PERFORMANCE:")
    print(f"   🕐 Totale tijd: {total_time:.1f} seconden")
    print(f"   🚀 Gemiddelde snelheid: {len(all_files)/total_time:.1f} bestanden/sec")
    print(f"   ⚡ Versnelling t.o.v. sequentieel: ~{MAX_WORKERS}x")
    print(f"{'='*80}")

if __name__ == "__main__":
    convert_html_to_markdown_parallel()

# Markdown cleaning

Het omzetten van HTML naar Markdown zorgde ervoor dat oa. haakjes niet correct stonden. Dit is hier grotendeels mee opgelost. Er zullen nog edgcases zitten en de vormgeving gaat allicht niet 100 procent correct zijn.

In [ ]:
import os
import re

# Definieer de bron- en doelmappen
SOURCE_MD_DIR = os.path.join('organized_documents_v3', '0_Type_Decreet', 'Markdown_Bestanden')
TARGET_MD_DIR = os.path.join('organized_documents_v3', '0_Type_Decreet', 'Markdown_Bestanden_Cleaned')

def clean_markdown_formatting(content):
    """
    Zet de vormgeving van artikelen recht in Markdown-bestanden.
    
    Transformeert alle varianten van artikelnummering:
    - Artikel X.
    - Artikel Xbis, Artikel Xter, etc.
    - Artikel X/Y (met slash)
    - Artikel Xbis/Y
    - Artikel Xter decies (met spatie)
    - Artikel Xquater decies
    
    En ruimt ook losse haakjes-structuren op.
    """
    
    # Uitgebreide Latijnse suffixen
    latin_suffixes = r'(?:bis|ter|quater|quinquies|sexies|septies|octies|nonies|decies|undecies|duodecies)'
    
    # STAP 1: Complexe pattern die alle mogelijke artikel-varianten ondersteunt
    article_pattern = rf'''
        (Artikel\s+                           # "Artikel "
         \d+                                  # Basis nummer (bijv. "17")
         (?:{latin_suffixes})?                # Optioneel eerste Latijns suffix (bijv. "bis")
         (?:/\d+)?                           # Optioneel slash + nummer (bijv. "/1")
         (?:\s+{latin_suffixes})?            # Optioneel spatie + tweede Latijns suffix (bijv. " decies")
         \.)                                 # Punt
         \s*\n+\s*                           # Whitespace en newlines
         \(                                  # Opening haakje
         \s*\n+                              # Whitespace en newlines
         (.*?)                               # Eerste deel (datum of ...)
         \n+\s*                              # Whitespace en newlines
         -                                   # Streepje
         \s*\n+                              # Whitespace en newlines
         (.*?)                               # Tweede deel (meestal ...)
         \n+\s*                              # Whitespace en newlines
         \)                                  # Sluitend haakje
    '''
    
    def replace_article(match):
        article_title = match.group(1)  # Volledige artikel titel
        first_part = match.group(2).strip()  # Eerste deel tussen haakjes
        second_part = match.group(3).strip()  # Tweede deel na de streep
        
        # Maak de nieuwe formatting
        return f"**{article_title}**({first_part}-{second_part})"
    
    # Vervang alle artikel-matches
    cleaned_content = re.sub(article_pattern, replace_article, content, flags=re.DOTALL | re.MULTILINE | re.VERBOSE)
    
    # STAP 2: Ruim losse haakjes-structuren op (die niet onder een artikel staan)
    # Pattern voor losse haakjes-structuren
    loose_brackets_pattern = r'''
        (?<!\*\*)                            # Niet voorafgegaan door ** (dus niet al verwerkt)
        \n\s*                                # Newline + optionele whitespace
        \(                                   # Opening haakje
        \s*\n+                               # Whitespace en newlines
        (.*?)                                # Eerste deel (datum of ...)
        \n+\s*                               # Whitespace en newlines
        -                                    # Streepje
        \s*\n+                               # Whitespace en newlines
        (.*?)                                # Tweede deel (meestal ...)
        \n+\s*                               # Whitespace en newlines
        \)                                   # Sluitend haakje
    '''
    
    def replace_loose_brackets(match):
        first_part = match.group(1).strip()
        second_part = match.group(2).strip()
        return f"\n({first_part}-{second_part})"
    
    # Vervang losse haakjes-structuren
    cleaned_content = re.sub(loose_brackets_pattern, replace_loose_brackets, cleaned_content, flags=re.DOTALL | re.MULTILINE | re.VERBOSE)
    
    return cleaned_content

def process_markdown_file(source_path, target_path):
    """Verwerkt een enkel Markdown-bestand."""
    try:
        # Lees het originele bestand
        with open(source_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Pas de opmaak aan
        cleaned_content = clean_markdown_formatting(content)
        
        # Schrijf naar het doelbestand
        with open(target_path, 'w', encoding='utf-8') as f:
            f.write(cleaned_content)
        
        return True, "Succesvol verwerkt"
    
    except Exception as e:
        return False, f"Fout: {str(e)}"

def copy_and_clean_markdown_files():
    """
    Kopieert alle Markdown-bestanden en zet de vormgeving recht.
    """
    
    if not os.path.exists(SOURCE_MD_DIR):
        print(f"FOUT: Bronmap '{SOURCE_MD_DIR}' niet gevonden.")
        return
    
    # Maak doelmap aan
    os.makedirs(TARGET_MD_DIR, exist_ok=True)
    print(f"Doelmap '{TARGET_MD_DIR}' aangemaakt/geverifieerd.")
    
    # Submappen
    submaps = ['Met_Annotaties', 'Zonder_Annotaties']
    
    total_processed = 0
    total_errors = 0
    total_article_replacements = 0
    total_bracket_replacements = 0
    
    for submap in submaps:
        source_subdir = os.path.join(SOURCE_MD_DIR, submap)
        target_subdir = os.path.join(TARGET_MD_DIR, submap)
        
        if not os.path.exists(source_subdir):
            print(f"Waarschuwing: Submap '{source_subdir}' niet gevonden. Overgeslagen.")
            continue
        
        # Maak doelsubmap aan
        os.makedirs(target_subdir, exist_ok=True)
        
        print(f"\nVerwerken van submap: {submap}")
        print(f"Bron: {source_subdir}")
        print(f"Doel: {target_subdir}")
        
        submap_processed = 0
        submap_errors = 0
        submap_article_replacements = 0
        submap_bracket_replacements = 0
        
        # Verwerk alle .md bestanden in de submap
        for filename in sorted(os.listdir(source_subdir)):
            if filename.endswith('.md'):
                source_file = os.path.join(source_subdir, filename)
                target_file = os.path.join(target_subdir, filename)
                
                # Tel vervangingen voor dit bestand
                with open(source_file, 'r', encoding='utf-8') as f:
                    original_content = f.read()
                
                # Test artikel-matches
                latin_suffixes = r'(?:bis|ter|quater|quinquies|sexies|septies|octies|nonies|decies|undecies|duodecies)'
                article_test_pattern = rf'''
                    (Artikel\s+
                     \d+
                     (?:{latin_suffixes})?
                     (?:/\d+)?
                     (?:\s+{latin_suffixes})?
                     \.)
                     \s*\n+\s*
                     \(
                     \s*\n+
                     (.*?)
                     \n+\s*
                     -
                     \s*\n+
                     (.*?)
                     \n+\s*
                     \)
                '''
                article_matches = re.findall(article_test_pattern, original_content, flags=re.DOTALL | re.MULTILINE | re.VERBOSE)
                
                # Test losse haakjes-matches
                bracket_test_pattern = r'''
                    (?<!\*\*)
                    \n\s*
                    \(
                    \s*\n+
                    (.*?)
                    \n+\s*
                    -
                    \s*\n+
                    (.*?)
                    \n+\s*
                    \)
                '''
                bracket_matches = re.findall(bracket_test_pattern, original_content, flags=re.DOTALL | re.MULTILINE | re.VERBOSE)
                
                file_article_replacements = len(article_matches)
                file_bracket_replacements = len(bracket_matches)
                
                success, message = process_markdown_file(source_file, target_file)
                
                if success:
                    submap_processed += 1
                    submap_article_replacements += file_article_replacements
                    submap_bracket_replacements += file_bracket_replacements
                    
                    if file_article_replacements > 0 or file_bracket_replacements > 0:
                        print(f"  ✓ {filename} ({file_article_replacements} artikelen, {file_bracket_replacements} losse haakjes)")
                    else:
                        print(f"  ✓ {filename}")
                else:
                    submap_errors += 1
                    print(f"  ✗ {filename} - {message}")
        
        print(f"  Resultaat: {submap_processed} succesvol, {submap_errors} fouten")
        print(f"  Aangepast: {submap_article_replacements} artikelen, {submap_bracket_replacements} losse haakjes")
        
        total_processed += submap_processed
        total_errors += submap_errors
        total_article_replacements += submap_article_replacements
        total_bracket_replacements += submap_bracket_replacements
    
    # Eindrapport
    print(f"\n{'='*60}")
    print(f"MARKDOWN OPSCHONING VOLTOOID")
    print(f"{'='*60}")
    print(f"📈 TOTAAL OVERZICHT:")
    print(f"   ✅ Succesvol verwerkt: {total_processed}")
    print(f"   ❌ Fouten: {total_errors}")
    print(f"   🔄 Artikelen aangepast: {total_article_replacements}")
    print(f"   📋 Losse haakjes aangepast: {total_bracket_replacements}")
    print(f"   📁 Totaal bestanden: {total_processed + total_errors}")
    print(f"   📂 Opgeslagen in: {TARGET_MD_DIR}")



if __name__ == "__main__":
    copy_and_clean_markdown_files()